In [1]:
import os
import random
import shutil
import pandas as pd
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


2024-12-02 12:07:18.924892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 12:07:19.081699: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#DATA PREPARATION
# File paths
data_folder = "data"
train_folder = "train"
test_folder = "test"
csv_file_path = 'dataset.csv'

# Create train and test directories
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Read the dataset
dataset = pd.read_csv(csv_file_path)

# Group images by label (person name)
grouped = dataset.groupby("label")

# Dictionary to store the count of images for each person
image_counts = {}

for label, group in grouped:
    # Get all image filenames for this person
    images = group["id"].tolist()
    
    # Count the number of images for this label
    image_counts[label] = len(images)
    
    # Shuffle the images for randomness
    random.shuffle(images)
    
    # Split images: 80% to train, remaining to test
    split_idx = int(0.8 * len(images))
    train_images = images[:split_idx]
    test_images = images[split_idx:]
    
    # Create label-specific folders in train and test
    train_person_folder = os.path.join(train_folder, label)
    test_person_folder = os.path.join(test_folder, label)
    os.makedirs(train_person_folder, exist_ok=True)
    os.makedirs(test_person_folder, exist_ok=True)
    
    # Move 80% images to train folder
    for img in train_images:
        src = os.path.join(data_folder, img)
        dst = os.path.join(train_person_folder, img)
        if os.path.exists(src):
            shutil.copy(src, dst)
    
    # Move the remaining 20% to the test folder
    for img in test_images:
        src = os.path.join(data_folder, img)
        dst = os.path.join(test_person_folder, img)
        if os.path.exists(src):
            shutil.copy(src, dst)

# Find the person with the largest and smallest number of images
largest_person = max(image_counts, key=image_counts.get)
smallest_person = min(image_counts, key=image_counts.get)

# Display the results
print("\nImage Counts for Each Person:")
for label, count in image_counts.items():
    print(f"{label}: {count}")

print(f"\nPerson with the largest number of images: {largest_person} ({image_counts[largest_person]} images)")
print(f"Person with the smallest number of images: {smallest_person} ({image_counts[smallest_person]} images)")

print("\nData preparation complete!")


Image Counts for Each Person:
Akshay Kumar: 50
Alexandra Daddario: 92
Alia Bhatt: 79
Amitabh Bachchan: 74
Andy Samberg: 92
Anushka Sharma: 68
Billie Eilish: 98
Brad Pitt: 120
Camila Cabello: 87
Charlize Theron: 78
Claire Holt: 96
Courtney Cox: 80
Dwayne Johnson: 61
Elizabeth Olsen: 71
Ellen Degeneres: 75
Henry Cavill: 106
Hrithik Roshan: 101
Hugh Jackman: 112
Jessica Alba: 108
Kashyap: 30
Lisa Kudrow: 70
Margot Robbie: 72
Marmik: 32
Natalie Portman: 105
Priyanka Chopra: 102
Robert Downey Jr: 113
Roger Federer: 77
Tom Cruise: 58
Vijay Deverakonda: 115
Virat Kohli: 49
Zac Efron: 91

Person with the largest number of images: Brad Pitt (120 images)
Person with the smallest number of images: Kashyap (30 images)

Data preparation complete!


In [3]:
imagesize_target=224
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(imagesize_target, imagesize_target, 3))

2024-12-02 12:07:23.720981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-02 12:07:23.774208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-02 12:07:23.774636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-02 12:07:23.775900: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable the

In [4]:
# Data generators for train and test sets
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(imagesize_target, imagesize_target),
    batch_size=32,  # You can adjust this based on your system's memory
    class_mode=None,  # No labels required for feature extraction
    shuffle=False  # Keep order consistent for mapping labels later
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(imagesize_target, imagesize_target),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

Found 2537 images belonging to 31 classes.
Found 1261 images belonging to 31 classes.


In [5]:
class_indices = train_generator.class_indices
inverse_class_indices = {v: k for k, v in class_indices.items()}  # Reverse the dictionary
joblib.dump(inverse_class_indices, "class_mapping.pkl")
print("Class mapping saved!")

Class mapping saved!


In [6]:
# Extract features for train set
print("Extracting features for training set...")
train_features = resnet_model.predict(train_generator, verbose=1)
train_labels = train_generator.classes  # Class indices corresponding to features
np.save("train_features.npy", train_features)  # Save features
np.save("train_labels.npy", train_labels)  # Save labels
print("Training features saved!")

Extracting features for training set...


2024-12-02 12:07:28.811431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8902


80/80 [==============================] - 11s 95ms/step
Training features saved!


In [7]:
# Extract features for test set
print("Extracting features for testing set...")
test_features = resnet_model.predict(test_generator, verbose=1)
test_labels = test_generator.classes  # Class indices corresponding to features
np.save("test_features.npy", test_features)  # Save features
np.save("test_labels.npy", test_labels)  # Save labels
print("Testing features saved!")

Extracting features for testing set...
40/40 [==============================] - 5s 113ms/step
Testing features saved!


In [8]:
# Optionally, perform Global Average Pooling to flatten features
gap = GlobalAveragePooling2D()
train_features_flat = gap(train_features).numpy()  # Shape: (num_train_samples, 2048)
test_features_flat = gap(test_features).numpy()    # Shape: (num_test_samples, 2048)

# Save flattened features
np.save("train_features_flat.npy", train_features_flat)
np.save("test_features_flat.npy", test_features_flat)
print("Flattened features saved!")

Flattened features saved!


In [ ]:
# Step 1: Load Saved Features and Labels
print("Loading features and labels...")
train_features = np.load("train_features_flat.npy")  # Flattened training features
test_features = np.load("test_features_flat.npy")    # Flattened testing features
train_labels = np.load("train_labels.npy")           # Training labels
test_labels = np.load("test_labels.npy")             # Testing labels
print("Features and labels loaded successfully!")

In [ ]:
# Step 2: Train the SVM Classifier
print("Training SVM classifier...")
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear', probability=True))
svm_classifier.fit(train_features, train_labels)
print("SVM training complete!")

In [ ]:
# Step 3: Evaluate the SVM Classifier
print("Evaluating the classifier...")
test_predictions = svm_classifier.predict(test_features)
# Calculate and display accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Detailed classification report
print("Classification Report:")
print(classification_report(test_labels, test_predictions))

# Step 4: Save the Trained SVM Model
print("Saving the trained SVM model...")
joblib.dump(svm_classifier, "svm_classifier.pkl")
resnet_model.save("resnet_model.h5")
print("SVM model saved as 'svm_classifier.pkl'.")


In [ ]:
# Step 1: Load Test Features and Labels
print("Loading test features and labels...")
test_features = np.load("test_features_flat.npy")  # Flattened test features
test_labels = np.load("test_labels.npy")          # Corresponding labels
print(f"Test features shape: {test_features.shape}")
print(f"Test labels shape: {test_labels.shape}")

# Step 2: Load the Trained SVM Model
print("Loading the trained SVM model...")
svm_classifier = joblib.load("svm_classifier.pkl")
print("SVM model loaded successfully!")

In [ ]:
# Step 3: Make Predictions on Test Features
print("Making predictions on the test dataset...")
test_predictions = svm_classifier.predict(test_features)

# Step 4: Evaluate the Model
# Calculate accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Generate classification report
print("Classification Report:")
print(classification_report(test_labels, test_predictions))

# Generate confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:


# Step 5: Visualize the Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=svm_classifier.classes_, yticklabels=svm_classifier.classes_)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# Step 6: Test on Individual Images
# Load the saved ResNet model
if not os.path.exists("resnet_model.h5"):
    raise FileNotFoundError("The ResNet model file 'resnet_model.h5' was not found.")
resnet_model = load_model("resnet_model.h5")
print("ResNet model loaded successfully!")

# Load the saved SVM model
if not os.path.exists("svm_classifier.pkl"):
    raise FileNotFoundError("The SVM model file 'svm_classifier.pkl' was not found.")
svm_classifier = joblib.load("svm_classifier.pkl")
print("SVM model loaded successfully!")

# Load the class mapping
if not os.path.exists("class_mapping.pkl"):
    raise FileNotFoundError("The class mapping file 'class_mapping.pkl' was not found.")
inverse_class_indices = joblib.load("class_mapping.pkl")
print("Class mapping loaded successfully!")

# Define function to predict a single image
def predict_single_image(img_path, resnet_model, svm_model, class_mapping):
    """
    Predict the label for a single image and return the class name.
    :param img_path: Path to the image file.
    :param resnet_model: Pre-trained ResNet model.
    :param svm_model: Trained SVM classifier.
    :param class_mapping: Dictionary mapping numeric labels to class names.
    :return: Predicted class name.
    """
    # Load and preprocess the image
    img = load_img(img_path, target_size=(imagesize_target, imagesize_target))  # Resize to ResNet input size
    img_array = img_to_array(img)  # Convert to NumPy array
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))  # Add batch dimension and preprocess
    
    # Extract features using ResNet
    print(f"Extracting features for image: {img_path}")
    image_features = resnet_model.predict(img_array)  # Extract features
    image_features_flat = image_features.mean(axis=(1, 2))  # Global Average Pooling

    # Predict label using SVM
    print("Predicting label...")
    numeric_label = svm_model.predict(image_features_flat)[0]
    class_name = class_mapping[numeric_label]  # Map numeric label to class name
    return class_name, img

# Example: Predict and display an individual image
img_path = "b.png"  # Replace with the path to a test image

try:
    # Predict the label
    predicted_class_name, img = predict_single_image(img_path, resnet_model, svm_classifier, inverse_class_indices)
    print(f"The predicted name for the image '{img_path}' is: {predicted_class_name}")
    
    # Display the image
    plt.imshow(img)
    plt.axis('off')  # Hide axis
    plt.title(f"Predicted: {predicted_class_name}")
    plt.show()
except Exception as e:
    print(f"Error during prediction: {e}")
